In [52]:
import numpy as np
import torch
import torch.nn as nn

# 一个线性回归的模型
构造一组数据x何其对应的标签y

## 使用cpu进行训练

In [53]:
x_values=[i for i in range(11)]
x_train = np.array(x_values, dtype=np.float32)
x_train = x_train.reshape(-1,1) # 将array转换为矩阵
x_train.shape

(11, 1)

In [54]:
y_values=[2*i +1 for i in x_values]
y_train = np.array(x_values, dtype=np.float32)
y_train = y_train.reshape(-1,1)
y_train.shape

(11, 1)

构建一个线性回归的模型. 线性回归就是一个不加激活函数的全连接层.

In [55]:
# 定义一个类集成nn.Model类
class LinearRegressionModel(nn.Module):
    def __init__(self,input_dim,output_dim):
        super(LinearRegressionModel,self).__init__()
        self.linear=nn.Linear(input_dim,output_dim) # 全连接层, 输入和输出的长度, 下面定义的都是1
    def forward(self,x):
        out = self.linear(x)
        return out

In [56]:
input_dim = 1
output_dim = 1
model = LinearRegressionModel(input_dim,output_dim)
# 这里的模型就是一个简单的全连接层
model

LinearRegressionModel(
  (linear): Linear(in_features=1, out_features=1, bias=True)
)

指定好参数和损失函数

In [57]:
epochs = 1000 # 迭代1000次
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate) # 回归优化器, parameters就是优化的参数
criterion = nn.MSELoss(); # 损失函数的类型, 这里用的是交叉熵

开始训练模型

In [58]:
for epoch in range(epochs):
    epoch+=1
    # 将行转换成tensor
    inputs = torch.from_numpy(x_train)
    labels = torch.from_numpy(y_train)
    
    # 每一次迭代的时候梯度清零
    optimizer.zero_grad()
    
    # 向前传播一次
    outputs = model.forward(inputs)
    
    # 使用损失函数方程计算损失函数
    loss = criterion(outputs,labels)
    
    # 反向传播, 就是进行一次求导过程
    loss.backward()
    
    # 更新权重参数, 准备进行重新计算
    optimizer.step()
    
    # 输出一次
    if epoch % 50 == 0:
        print("Epoch {} , loss {}".format(epoch,loss.item()))

Epoch 50 , loss 0.0896872952580452
Epoch 100 , loss 0.05115422233939171
Epoch 150 , loss 0.02917645126581192
Epoch 200 , loss 0.016641169786453247
Epoch 250 , loss 0.009491496719419956
Epoch 300 , loss 0.005413589999079704
Epoch 350 , loss 0.0030877008102834225
Epoch 400 , loss 0.0017611081711947918
Epoch 450 , loss 0.001004478894174099
Epoch 500 , loss 0.0005729169934056699
Epoch 550 , loss 0.0003267706779297441
Epoch 600 , loss 0.0001863776269601658
Epoch 650 , loss 0.00010630281758494675
Epoch 700 , loss 6.062987813493237e-05
Epoch 750 , loss 3.4582022635731846e-05
Epoch 800 , loss 1.972367499547545e-05
Epoch 850 , loss 1.1249919225519989e-05
Epoch 900 , loss 6.416294581867987e-06
Epoch 950 , loss 3.6596031804947415e-06
Epoch 1000 , loss 2.0872867025900632e-06


预测模型的结果

In [59]:
predicted = model(torch.from_numpy(x_train).requires_grad_()).data.numpy()
predicted

array([[2.6875865e-03],
       [1.0023006e+00],
       [2.0019135e+00],
       [3.0015266e+00],
       [4.0011396e+00],
       [5.0007524e+00],
       [6.0003653e+00],
       [6.9999785e+00],
       [7.9995914e+00],
       [8.9992046e+00],
       [9.9988174e+00]], dtype=float32)

保存模型到本地, 保存的是一个字典, 这里指保存参数

In [60]:
torch.save(model.state_dict(),'./models/lr_model.pkl')

读取模型

In [61]:
model.load_state_dict(torch.load('./models/lr_model.pkl'))


<All keys matched successfully>

## 使用gpu进行训练

In [62]:
if torch.cuda.is_available():
    class LinearRegessionModel(nn.Module):
        def __init__(self,input_dim,output_dim):
            super(LinearRegressionModel,self).__init__()
            self.Linear=nn.Linear(input_dim,output_dim)
            
        def forward(self,x):
            out=self.linear(x)
            return out
    input_dim=1
    output_dim=1
    model=LinearRegressionModel(input_dim,output_dim)
    device=torch.device("cuda:0"if torch.cuda.is_available() else "cpu") #如果GPU配置好了，那就用GPU；否则用CPU
    model.to(device) #将模型放入到GPU中去
    crierion=nn.MSELoss()
    learning_rate=0.01
    optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)
    #将数据传入GPU
    epochs=1000
    for epoch in range(epochs):
        epoch+=1
        inputs=torch.from_numpy(x_train).to(device) # 同样将所有需要使用的都放入gpu
        outputs=torch.from_numpy(y_train).to(device)
        
        optimizer.zero_grad()
        
        outputs=model(inputs)
        loss=criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        if epoch %50==0:
            print("epoch {},loss {}".format(epoch,loss.item()))